In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.insert(0, "../../../scirpy/")

import scirpy as ir
import scanpy as sc
import random
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'scanpy'

In [ ]:
adata.shape

In [ ]:
adata.obs["TRA_1_cdr3"].unique().size

In [ ]:
frac_unique_alpha = adata.obs["TRA_1_cdr3"].unique().size / adata.shape[0]

In [ ]:
frac_unique_beta = adata.obs["TRB_1_cdr3"].unique().size / adata.shape[0]